## Corridors Crash Data

### 1. Needs number of crashes, fatalities, injuries, severity categorization, Mode for each corridor in study area
### 2. Database: DOTDEVRHPGSQL01
### 3. Tables: study_area, nysdot_all, fatality_nycdot_current, lion

In [2]:
from ris import db2  #library designed for SQL database connection and querying
from IPython.display import clear_output
import datetime 
from datetime import date
import pandas as pd
import os


clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
print 'Notebook run: {}'.format(timestamp)
print os.getcwd()
# %load_ext sql 

Notebook run: 2019-08-06 15:54
E:\RIS\Staff Folders\Samuel\Requests\SH\Crash_Data


In [3]:
#Database connections
#gdb = db2.PostgresDb('dotdevpgsql02', 'GISGRID', quiet = True)
cdb = db2.PostgresDb('DOTDEVRHPGSQL01', 'CRASHDATA', quiet = True)

User name (CRASHDATA):soge
Password (CRASHDATA)········


In [18]:
df = pd.read_excel('Corridors List_20190725.xlsx', sheet_name='Crash Database  ')
#df

In [19]:
cols=df.columns[8:18]

In [20]:
new_headers=map(str, cols)
#new_headers

In [21]:
new_headers.insert(0,str(df.columns[1]))

In [22]:
corridors_crash_data = (db2.query_to_table(cdb, """

                --2. Query 2 including all data except fatality Edited  

                select id_b, 
                sum(tot_crashes) tot_crashes, 
                sum(tot_inj) tot_inj, 
                sum(a) a,
                sum(b) b,
                sum(c) c,
                sum(ksi) ksi,
                sum(ped) ped,
                sum(bike) bike 



                from (

                --Selecting sum of all midblock injuries within study area grouped by id_b

                select id_b, 
                count(case_num) tot_crashes, 
                sum(num_of_inj) tot_inj ,
                sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text))) AS A,
                sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'B'::text, ''::text))) AS B,
                sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'C'::text, ''::text))) AS C,
                sum(length(ext_of_inj::text)) AS KSI,
                sum(case when accd_type_int= 1 then num_of_inj else 0 end) as Ped,
                sum(case when accd_type_int= 2 then num_of_inj else 0 end) as Bike

                from(

                select distinct id_b, case_num, ext_of_inj, crashid, num_of_inj, accd_type_int, loc from (
                select * from nysdot_all where case_yr between 2005 and 2014 and exclude = 0) nys
                join(
                    select id_b, segmentid 
                    from(select mft,segmentid, geom from lion 
                         where mft in (select l.mft from lion l join working.study_area sa on l.segmentid = sa.segmentid)) ll

                    join(select sa.id_b, l.mft 
                         from lion l join working.study_area sa
                         on l.segmentid = sa.segmentid 
                         group by id_b, l.mft) id
                         on ll.mft = id.mft) d

                on nys.segmentid = d.segmentid
                where id_b is not null
                )corr
                group by id_b


                union all

                --Selecting sum of all itx injuries within study area grouped by id_b

                select id_b, 
                count(case_num) tot_crashes, 
                sum(num_of_inj) tot_inj ,
                sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text))) AS A,
                sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'B'::text, ''::text))) AS B,
                sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'C'::text, ''::text))) AS C,
                sum(length(ext_of_inj::text)) AS KSI,
                sum(case when accd_type_int= 1 then num_of_inj else 0 end) as Ped,
                sum(case when accd_type_int= 2 then num_of_inj else 0 end) as Bike

                from(

                select distinct id_b, case_num, ext_of_inj, crashid, num_of_inj, accd_type_int, loc 
                from (select * from nysdot_all where case_yr between 2005 and 2014 and exclude = 0) nys
                join (
                    select mids.id_b,t.mid 
                    from(select masteridfrom mid from lion 
                         where mft in (select l.mft from lion l join working.study_area sa on l.segmentid = sa.segmentid)
                         union 
                         select masteridto mid from lion 
                         where mft in (select l.mft from lion l join working.study_area sa on l.segmentid = sa.segmentid)) t

                    join(select id_b, l.masteridfrom mid
                         from lion l 
                         join working.study_area sa on l.segmentid = sa.segmentid
                         union
                         select id_b, l.masteridto mid
                         from lion l 
                         join working.study_area sa on l.segmentid = sa.segmentid) mids	
                        on t.mid = mids.mid) mm

                on nys.masterid::int = mm.mid
                where id_b is not null
                ) itx
                group by id_b

                ) corr_itx
                --where id_b = '17AM'
                group by id_b

                """ 
                        ))

In [23]:
corridors_crash_data.drop(corridors_crash_data.tail(1).index,inplace=True)

In [24]:
corridors_crash_data

,id_b,tot_crashes,tot_fat,tot_inj,a,b,c,ksi,unk,ped,bike
0,14ABK,15,8.0,17,1,1,15,17,0,1,0
1,14ABX,10,0.0,11,2,1,8,11,0,6,0
2,14AM,0,0.0,0,0,0,0,0,0,0,0
3,14AQ,1,0.0,1,0,0,1,1,0,0,0
4,14ASI,0,4.0,0,0,0,0,0,0,0,0
5,14BBK,219,39.0,181,17,21,142,210,1,18,6
6,14BBX,186,0.0,166,7,16,140,178,0,35,6
7,14BM,287,0.0,243,26,47,167,250,2,102,32
8,14BQ,101,8.0,98,4,10,82,99,1,20,1
9,14BSI,31,0.0,30,0,3,27,30,0,2,0


In [25]:
headers=map(str, corridors_crash_data.columns)
#headers

In [26]:
dictionary = [{i:j} for i,j in zip(headers, new_headers)]
for i in dictionary:        
    corridors_crash_data=corridors_crash_data.rename(columns=i)

#corridors_crash_data.columns
#corridors_crash_data

In [27]:
#pd.merge(df.loc[:,df.columns[:8]],corridors_crash_data, how='left').shape
data = df.loc[:, df.columns[:8]].merge(corridors_crash_data, how='left')

In [31]:
data.to_csv('data-{}.csv'.format(datetime.datetime.now().strftime('%Y%m%d')),index=False) 